# 8. 데이터 준비하기: 조인, 병합, 변형

In [1]:
import numpy as np
import pandas as pd

In [3]:
left = pd.DataFrame({
    'id':[1,2,3,4,5],
    'name':['a','b','c','d','e'],
    'subject_id':['sub1','sub2','sub3','sub4','sub5']
})
right = pd.DataFrame({
    'id':[1,2,3,4,5],
    'name':['a1','b1','c1','d1','e1'],
    'subject_id':['sub2','sub4','sub3','sub6','sub5']
})

In [4]:
df = pd.merge(left,right, on='id') # SELECT * FROM WHERE left.id = right.id
df

,id,name_x,subject_id_x,name_y,subject_id_y
0,1,a,sub1,a1,sub2
1,2,b,sub2,b1,sub4
2,3,c,sub3,c1,sub3
3,4,d,sub4,d1,sub6
4,5,e,sub5,e1,sub5


In [5]:
df2 = pd.merge(left,right, on=['id','subject_id']) # 조인조건 2개이상 '[]'
df2

,id,name_x,subject_id,name_y
0,3,c,sub3,c1
1,5,e,sub5,e1


In [6]:
# left out join
df3 = pd.merge(left,right, on='subject_id', how='right') 
# how='' : 조인유형 left, right, inner, outer
df3

,id_x,name_x,subject_id,id_y,name_y
0,2.0,b,sub2,1,a1
1,3.0,c,sub3,3,c1
2,4.0,d,sub4,2,b1
3,5.0,e,sub5,5,e1
4,NaN,NaN,sub6,4,d1


### 8.2. 데이터 합치기

- pandas.merge 하나 이상의 키를 기준으로 df의 로우를 합친다. SQL이나 다른 RDB의 join연산과 유사
- pandas.concat 하나의 축을 따라 객체를 이어붙인다.

#### 8.2.1. 데이터베이스 스타일로 DF합치기
- 병합이나 조인 연산은 관계형 데이터베이스의 핵심적인 연산인데, 하나 이상의 키를 사용해서 데이터 집합의 로우를 합친다.

In [8]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],
                    'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],
                    'data2':range(3)})
print(df1)
print(df2)

key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


In [9]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [10]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [11]:
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],
                    'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a','b','d'],'data2':range(3)})
pd.merge(df3, df4, left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


- how인자로 left, right, outer를 넘겨서 각각 왼쪽조인, 오른쪽조인, 외부조인을 수행할 수도 있다.

In [12]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


### 다대다 병합
- 다대다 조인은 두 로우의 데카르트 곱을 반환한다.
- 조인 메서드는 결과에 나타나는 구별되는 키에 대해서만 적용한다.
- 여러개의 키를 병합하려면 컬럼이름이 담긴 리스트를 넘긴다.

In [14]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','b'], 'data1':range(6)})
df2 = pd.DataFrame({'key':['a','b','a','b','d'], 'data2': range(5)})

In [15]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [16]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [17]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [18]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [20]:
left = pd.DataFrame({
    'key1':['foo','foo','bar'],
    'key2':['one','two','one'],
    'lval':[1,2,3]
})
right = pd.DataFrame({
    'key1':['foo','foo','bar','bar'], 
    'key2':['one','one','one','two'],
    'rval':[4,5,6,7]
})
pd.merge(left, right, on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [21]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [22]:
pd.merge(left, right, on='key1', suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7
